In [1]:
import json
from datetime import datetime
from tqdm import tqdm
from shroom_classifier_lcel_baseline_v4_persona import ShroomClassifier

In [2]:
def run_experiment(cp, reference, submission):
    data = json.load(open(reference, "r"))
    predictions = []
    for dp in tqdm(data):
        prediction = cp.classify(dp)
        predictions.append(prediction)
    json.dump(predictions, open(submission, "w+"))

In [3]:
RUN = "baseline_v4_persona"
MODEL = "gpt-3.5-turbo"
TEMP = 1.2
DATE = datetime.utcnow().date().isoformat()

In [4]:
CLASSIFIER = ShroomClassifier(model_name=MODEL, temperature=TEMP)
run_experiment(CLASSIFIER, 'reference/val.model-agnostic.json', 'submission/val.model-agnostic.json')
run_experiment(CLASSIFIER, 'reference/val.model-aware.json', 'submission/val.model-aware.json')

/Users/bradleyallen/Documents/GitHub/shroom/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:115: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(
100%|██████████| 501/501 [05:54<00:00,  1.41it/s]


In [5]:
! python score.py --is_val submission reference scores/score-{MODEL}-{RUN}-{DATE}.txt
! mv submission/val.model-agnostic.json results/val.model-agnostic.{MODEL}-{RUN}-{DATE}
! mv submission/val.model-aware.json results/val.model-aware.{MODEL}-{RUN}-{DATE}